In [1]:
import torch

cpu_tensor=torch.zeros(4,3) #3개짜리 4개
device=torch.device("cuda:0") #device 선언
gpu_tensor=cpu_tensor.to(device)
print(gpu_tensor)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], device='cuda:0')


선형 회귀분석

주어진 데이터를 가장 잘 설명하는 직선 하나를 찾는 것.

어떻게? => 예측 값과 실제 결과값의 차이를 최소화 하는 방향으로 !

예측값과 실제 결과값의 차이 => LOSS,COST (손실함수, 비용함수)

MSE == L2 Loss

최소화 하는 방향? => Optimization

대표적으로 경사하강법

경사란 결국 기울기, 기울기란 결국 미분 값, 

얼마만큼 최소화 할래? => Learning Rate

Pytorch 결국 연산그래프를 만들고 경사를 계산하는 방식

데이터는 Tensor를 사용.

Tensor는 다차원 배열이라고 정의할 수 있음.




In [0]:
import torch

x=torch.Tensor(2,3) #3개짜리가 2개

print(x)

In [0]:
import torch

#data,type,device,requires_grad

x=torch.Tensor([[1,2,3],[1,3,2]])

print(x)
print(x.shape)

#torch.XX
#torch.cuda.XX
#XX => FloatTensor, DoubleTensor, IntTensor
#device는 텐서를 어느 기기에 올릴 것인지를 명시함.
#requires_grad는 이 텐서에 대한 기울기를 저장할지 여부를 결정

x_tensor=torch.tensor(data=[2.0,3.0],requires_grad=True)
print(x_tensor)



In [0]:
import torch

x=torch.tensor(data=[2.0,3.0],requires_grad=True)
z=2*(x**2)+3 #연산 그래프

target=torch.Tensor([3.0,4.0])

print(z)
print(z-target)
print(abs(z-target))
print(torch.abs(z-target))
print(torch.sum(torch.abs(z-target)))

loss=torch.sum(torch.abs(z-target))

loss.backward()

print(x.grad,z.grad)

In [0]:
import torch
import torch.nn as nn 
import torch.optim as optim #경사하강법
import torch.nn.init as init #텐서에 초기값을 주기 위한..

num_data=1000
num_epoch=500


x=init.uniform_(torch.Tensor(num_data,1),-10,10)#-10부터 10까지 균등하게(uniform)하게 초기화
noise=init.normal_(torch.FloatTensor(num_data,1),std=1) #노이즈, 현실성 반영, 표준 정규분포를 따르는 노이즈 = 가우시안 노이즈 , mean(평균)=0, std(표준편차)=1

y=2*x+3
y_noise=2*(x+noise)+3 

model=nn.Linear(1,1) #들어오는 특성의 수, 결과로 나오는 특성의 수 
loss_func=nn.L1Loss() #차이의 절대값의 평균

optimizer=optim.SGD(model.parameters(),lr=0.01)

label=y_noise

for i in range(num_epoch):
  optimizer.zero_grad() #기울기 0으로 초기화 , 기울기를 초기화 해야 새로운 가중치와 편차에 대해서 새로운 기울기를 구할 수 있음.
  output=model(x)
  
  loss=loss_func(output,label)
  loss.backward() #각 변수 즉 w,b에 대한 기울기가 계산됨.
  
  optimizer.step() #업데이트 
  
  if i%10==0:
    print(loss.data)
    param_list=list(model.parameters())
    print(param_list[0].item(), param_list[1].item())
    



여러 입력이 들어모면 각각 가중치를 곱해주고 편차도 더해준다.

이렇게 다 더한 값을 활성화 함수를 통해 변형하여 전달하는 단위를 인공뉴런이라고 한다.

인공신경망은 입력층, 은닉층, 출력층으로 이루어짐.

딥러닝 즉 심층 신경망은 은닉층이 2개이상인 것을 의미한다.



In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init


num_data=1000
num_epochs=10000


noise=init.normal_(torch.FloatTensor(num_data,1),std=1) #노이즈 더하기 , 모양은 x와 동일해야 

x=init.uniform_(torch.Tensor(num_data,1),-15,15) #균등분포 초기화 

y=(x**2)+3 #목적함수

y_noise=y+noise #목적 함수 즉 목표

model=nn.Sequential(
    nn.Linear(1,6),
    nn.ReLU(),
    nn.Linear(6,10),
    nn.ReLU(),
    nn.Linear(10,6),
    nn.ReLU(),
    nn.Linear(6,1)
)

loss_func=nn.L1Loss() #Loss는 L1 Loss를 이용한다.

optimizer=optim.SGD(model.parameters(),lr=0.0002) #옵티마이저는 SGD를 이용

loss_array=[] #loss 담기 위한

for i in range(num_epochs):
  optimizer.zero_grad()
  output=model(x) #forward 과정 , 즉 값을 예측
  loss=loss_func(output,y_noise) #예측값과 목표값의 loss 계산
  loss.backward() #미분 진행, 즉 기울기 계산 진행
  optimizer.step() #학습 진행 
  
  loss_array.append(loss)
            
import matplotlib.pyplot as plt

plt.plot(loss_array)
plt.show()


CNN이란 국소적인 영역을 보고 단순한 패턴에 자극을 받는 단순세포와 넓은 영역을 보고 복잡한 패턴에 작그을 받는 복잡세포가 레이어를 이루고 있다.

필터 = 커널

하나의 필터에 대해 이미지를 쭉 지나가면서 이미지의 부분 부분이 필터와 얼마나 일치하는지를 계산한다.

이동의 단위는 stride이다.

하나의 이미지에 대하여 여러개의 필터를 적용할 수 있다.

필터 하나당 입력 이미지 전체에 대한 필터의 일치정도가 나오는데 이를 활성화 지도, 특성지도라고 한다.

활성화 지도의 크기는 입력이미지와, 필터의 크기, 스트라이드에 크기에 따라서 결정이 된다.

floor((입력크기-커널크기/스트라이드크기)+1 )

문제는 입력이미지에 필터를 계속 적용하다보면 크기가 줄어듬....

이렇게 되면 인공신경망 입장에서는 은닉층의 개수에 제한이 생기는 것과 같고 결과적으로 복잡한 패턴을 학습할 수가 없게 됨.

따라서 패팅이라는 기법이 등장하게 됨.

패딩이란 일정한 크기의 층으로 이미지를 감싸는 것을 의미한다.

9x9에 패딩 1을 주면 11x11로 변하게 됨.
 
네 방향 테두리를 쭉 돌아가면서 패팅을 적용하기 때문에 이미지 크기는 1이 아니라 2가 늘어나게 됨.

floor((입력크기-커널크기+(2*패딩의크기)/스트라이드크기)+1 )


반대로 입력 이미지가 엄청나게 크다면 굳이 패딩이 필요없음,, 여전히 클 것이기에...

이럴 때 풀링이라는 것을 사용함.

풀링은 다운 샘플링 또는 서브 샘플링에 일종이며 맥스풀링과 평균 풀링이 있음.

풀링의 범위가 2X2라면 한번에 2칸씩 이동...

결론적으로 대게 몇번의 합성곱 연산을 활성화 함수와 함께 적용한 이후 풀링으로 전체 크기를 줄여주는 과정을 반복함.


분류 문제에서는 여러 클래스가 있음.

정답은 원 핫 인코딩으로 변환.

원핫 인코딩은 본래의 정답을 확률 분포로 변환해 주는 것.

즉 정답에 해당하는 확률은 1 아니면 0

신경망의 결과값과 정답을 비교해야 하니 신경망의 결과값도 확률의 형태여야

이래서 등장한 것이 소프트 맥스

소프트맥스란 결과값을 확률로 바꾸어주는 역할.

결과값 벡터에 있는 값들이 지수함수를 통과하면 모든 값이 양수로 되는 효과가 있음. 따라서 소프트 맥스 함수에 exp가 있는 것.

그렇다면 두개의 확률벡터는 어떤 기준으로 손실(LOSS)를 측정할 까?

L1 => 절대값 거리를 기준
L2 => 차이의 제곱을 기준으로 하는 평균제곱오차

교차앤트로피 : 목표로하는 최적의 확률분포 p와 이를 근사하려는 확류분포 q가 얼마나 다른지를 측정하는 방법

교차앤트로피는 예측이 잘못될 수록 L!손실보다 더 크게 증가.. 그만큼 패널티가 크고 손실값이 크기 때문에 학습면에서 장점 보유, 따라서 분류 문제에서 교차엔트로피 손실 많이 사용


하이퍼파라미터: 학습의 대상이 아니라 학습 이전에 정해놓은 변수

CNN - MNIST 분류

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader #데이터를 하나씩 전달하지 않고 원하는 배치사이즈대로 묶어서 전달하거나 효율적인 학습을 위한 셔플링 등을 여기서 해줌

batch_size=256
learning_rate=0.0002
num_epochs=10


#transform과 target_transform은 각각 데이터와 라벨에 대한 변형을 의미한다.
#여기서는 transform은 단순히 텐서로 변환... 라벨에 대한 변환은 안함.
#num_workes: 데이터를 묶을 때 프로세스 개수
#drop_last: 묶고 남는 데이터는 버릴지 여부
#train_loader와 test_loader는 순차적으로 모델에 데이터를 전달해준다.
mnist_train=dset.MNIST("./",train=True,transform=transforms.ToTensor(),target_transform=None,download=True) #학습 데이터
mnist_test=dset.MNIST("./",train=False,transform=transforms.ToTensor(),target_transform=None,download=True) #테스트 데이터

train_loader=torch.utils.data.DataLoader(mnist_train,batch_size=batch_size,shuffle=True,num_workers=2,drop_last=True)
test_loader=torch.utils.data.DataLoader(mnist_test,batch_size=batch_size,shuffle=False,num_workers=2,drop_last=True)


class CNN(nn.Module):
  
  def __init__(self):
    super(CNN,self).__init__() #CNN클래스의 부모 클래스인 nn.Module을 초기화 하는 역할.
    
    #합성곱 연산
    self.layer=nn.Sequential(
        
        #in_channels, out_channels,kernel_size,stride,padding 등이 잇음.
        #in_channels: RGB 3채널 데이터를 입력으로 받았던 것을 예로 들어 설명하면 in_channels는 3
        #out_channels: 필터의 개수
        
        #따라서 입력:  [batch_size, in_channels, 가로, 세로]
        #합성곱 연산의 결괴: [batch_size,out_channels,가로,세로]
        #MNIST입력 같은 경우는 [batch_size,1,가로,세로]
        
        
        
        nn.Conv2d(1,16,5), # (batch_size,out_channels=필터의 개수=커널의 개수,kernel_size) //  (stride =1  padding=0)으로 기본 설정 //  입력으로는 [batch_size,1,28,28]이 들어올 것.
        #위에 conv연산 통과시 [batch_size,16,24,24]가 될 것임. #16인 이유는 필터의 개수, 24는 연산에 따라서..
        nn.ReLU(),
        nn.Conv2d(16,32,5), #[batch_size,32,20,20]이 된다.
        nn.ReLU(),
        nn.MaxPool2d(2,2), #(kernel_size,stride) , 연산후에는 텐서가 반으로 줄어듬. [batch_size,32,10,10] //  kernel_size,stride,padding등을 받음. kernel_size는 풀링 연산을 할 때 한번에 훑는 영역의 크기, 
        nn.Conv2d(32,64,5), # [batch_size,64,6,6]
        nn.ReLU(),
        nn.MaxPool2d(2,2) # [batch_size,64,3,3]
    )
    
    self.fc_layer=nn.Sequential(
        nn.Linear(64*3*3,100),#batch_size,100
        nn.ReLU(),
        nn.Linear(100,10) #[batch_size,10]
    )
    
  def forward(self,x):
      out=self.layer(x) #convolution and max pooling
      out=out.view(batch_size,-1) #Fully connected 입력을 위하여 ...
      out=self.fc_layer(out)
      return out
    
  
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model=CNN().to(device)
loss_func=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

loss_arr=[]

for i in range(num_epochs):
  for j,[image,label] in enumerate(train_loader):
    x=image.to(device)
    y_=label.to(device)
    
    optimizer.zero_grad()
    output=model.forward(x)
    loss=loss_func(output,y_)
    loss.backward()
    optimizer.step()
    
    if j%1000 ==0:
      print(loss)
      loss_arr.append(loss.cpu().detach().numpy())
      
      
correct=0
total=0

with torch.no_grad():
  for image,label in test_loader:
    x=image.to(device)
    y_=label.to(device)
    
    output=model.forward(x)
    _,output_index=torch.max(output,1) #(최대값, 그 인덱스)
    
    total+=label.size(0) #size(0)이것은 배치사이즈에 해당함.
    correct+=(output_index==y_).sum().float()
    print("Accuracy of Test Data:{}".format(100*correct/total))


RNN

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

n_hidden=35 # 순환 신경망 노드 수
lr=0.01
epochs=1000

string="hello pytorch.how long can rnn cell number"
chars="abcdefghijklmnopqrstuvwxyz?!.,;:01 "

char_list=[i for i in chars]
n_letters=len(char_list)

def string_to_onehot(string):
  start=np.zeros(shape=len(char_list),dtype=int)
  end=np.zeros(shape=len(char_list),dtype=int)
  start[-2]=1
  end[-1]=1
  
  for i in string:
    idx=char_list.index(i)
    zero=np.zeros(shape=n_letters,dtype=int)
    zero[idx]=1
    start=np.vstack([start,zero])
  output=np.vstack([start,end])
  return output

def onehot_to_word(onehot_1):
  onehot=torch.Tensor.numpy(onehot_1)
  return char_list[onehot.argmax()]



class RNN(nn.Module):
  
  def __init__(self,input_size,hidden_size,output_size):
    super(RNN,self).__init__()
    
    self.input_size=input_size
    self.hidden_size=hidden_size
    self.output_size=output_size
    
    self.i2h=nn.Linear(input_size,hidden_size)
    self.h2h=nn.Linear(hidden_size,hidden_size)
    self.i2o=nn.Linear(hidden_size,output_size)
    self.act_fn=nn.Tanh()
    
  def forward(self,input,hidden):
    hidden=self.act_fn(self.i2h(input)+self.h2h(hidden))
    output=self.i2o(hidden)
    return output,hidden
  def init_hidden(self):
    return torch.zeros(1,self.hidden_size)
  
rnn=RNN(n_letters,n_hidden,n_letters)


loss_func=nn.MSELoss()
optimizer=torch.optim.Adam(rnn.parameters(),lr=lr)


one_hot=torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor())
print(one_hot.size())#torch.Size([44, 35])

for i in range(epochs):
  rnn.zero_grad()
  total_loss=0
  hidden=rnn.init_hidden()
  
  
  for j in range(one_hot.size()[0]-1): 
    input_=one_hot[j:j+1,:] #현재단어
    target=one_hot[j+1] #다음단어 예측
    
    output,hidden=rnn.forward(input_,hidden) #output과 hidden을 받음. 
    loss=loss_func(output.view(-1),target.view(-1)) #output과 target에 대한 LOSS
    total_loss+=loss#BPTT
    input_=output #출력을 다시 입력으로 
  total_loss.backward()
  optimizer.step()
  
  if i%10==0:
    print(total_loss)
    
    
start=torch.zeros(1,len(char_list))
start[:-2]=1

with torch.no_grad():
  hidden=rnn.init_hidden()
  input_=start
  output_string=""
  
  for i in range(len(string)):
    output,hidden=rnn.forward(input_,hidden)
    output_string+=onehot_to_word(output.data)
    input_=output #output이 그 다음 step 입력으로 들어간다.
    
  print(output_string)